In [6]:

!pip install helical

!pip install datasets --upgrade

  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 3.2.0
    Uninstalling datasets-3.2.0:
      Successfully uninstalled datasets-3.2.0


  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.20.0
    Uninstalling datasets-2.20.0:
      Successfully uninstalled datasets-2.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
helical 0.0.1a16 requires datasets==2.20.0, but you have datasets 3.2.0 which is incompatible.


In [2]:


from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import anndata as ad
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from scipy.sparse import lil_matrix
import torch.optim as optim
from helical.models.scgpt.model import scGPT, scGPTConfig
from helical.models.geneformer.model import Geneformer, GeneformerConfig
from copy import deepcopy
from torch.nn.functional import one_hot

INFO:datasets:PyTorch version 2.5.1+cu121 available.
INFO:datasets:Polars version 1.9.0 available.
INFO:datasets:Duckdb version 1.1.3 available.
INFO:datasets:TensorFlow version 2.17.1 available.
INFO:datasets:JAX version 0.4.33 available.
INFO:helical:Caduceus not available: If you want to use this model, ensure you have a CUDA GPU and have installed the optional helical[mamba-ssm] dependencies.


In [ ]:
from datasets import load_dataset
ds = load_dataset("helical-ai/yolksac_human",trust_remote_code=True, split="train[:65%]",download_mode="reuse_cache_if_exists")

Generating train split:   0%|          | 0/25344 [00:00<?, ? examples/s]

In [3]:
observation_columns = [obs for obs in list(ds.features.keys()) if not obs == 'raw_counts']
obs_data = pd.DataFrame(ds.select_columns(observation_columns).data.to_pandas(),columns=observation_columns)
lil = lil_matrix((len(ds),ds[0]['size']))
lil.data = np.array(ds['raw_counts'],dtype="object")
lil.rows = np.array(ds['rows'],dtype="object")
adata = ad.AnnData(lil.tocsr(),obs=obs_data)
adata.var_names = ds.features['raw_counts'].id.split(",")
adata.var['gene_name'] = adata.var_names.str.upper()

/usr/local/lib/python3.10/dist-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
# get labels: the celltype
num_types = adata.obs["LVL1"].unique().shape[0]
id2type = dict(enumerate(adata.obs["LVL1"].astype("category").cat.categories))

celltypes_labels = np.array(adata.obs["LVL1"].tolist())

In [1]:
device = "cuda" if torch.cuda.is_available() else "cpu"
scgpt_config = scGPTConfig(batch_size=50, device=device)
scgpt = scGPT(configurer = scgpt_config)
print(adata.var.index)


def knockout_genes(data,model,genes):
  """ Generate embeddings for the original dataset and modified dataset using
  the selected model. The modified dataset has gene expressions set to 0 for
  all genes present in the genes list.

  Inputs:
      data (anndata): dataset that is being considered
      model ()
  """
  # knockout genes using batching to reduce explosion in RAM usage
  batch_size = 200
  original_data = data.copy()
  print(len(genes))
  for i in range(0, len(genes), batch_size):
        print(i)
        batch_genes = genes[i:i+batch_size]
        # set expression values for genes in the current batch to 0
        for gene in batch_genes:
          if gene in data.var.index:
            gene_idx = data.var.index.get_loc(gene)  # Direct indexing
            data[:, gene_idx] = 0

  perturbed_data = data.copy()
  new_data = model.process_data(adata=perturbed_data, gene_names="gene_name")
  x_model_dropped = model.get_embeddings(new_data)
  print(len(x_model_dropped))
  # generate embeddings on original data
  new_data = model.process_data(adata = original_data, gene_names = "gene_name")
  x_model = model.get_embeddings(new_data)
  print(x_model, x_model_dropped)
  return x_model, x_model_dropped
#genes_to_knockout = ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M']
genes_to_knockout = list(adata.var_names[0:int(len(adata.var_names)/100)])

x_model, x_model_dropped = knockout_genes(adata,scgpt,genes_to_knockout)


NameError: name 'torch' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(x_model)
plt.plot(x_model_dropped)
plt.show()

In [ ]:
# mse
mse = 0
for i,j in zip(x_model,x_model_dropped):
  mse += (i,j)**2
  mse /= len(x_model)
print(mse)